In [10]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score



In [11]:
df = pd.read_csv("../data/train.csv")

train, validation = train_test_split(df, test_size=0.2, random_state=200, shuffle=True)

#X_train = df_train.drop(columns=["SalePrice"])
#y_train = df_train["SalePrice"]

In [13]:
# Load TF-DF
import tensorflow_decision_forests as tfdf

# Convert the dataset into a TensorFlow dataset.
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label="SalePrice", task=tfdf.keras.Task.REGRESSION)
validation_ds = tfdf.keras.pd_dataframe_to_tf_dataset(validation, label="SalePrice", task=tfdf.keras.Task.REGRESSION)

# Train a Random Forest model.
model = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.REGRESSION)
model.fit(train_ds)

# Summary of the model structure.
#model.summary()

# Assuming test_ds contains your test data
predictions = model.predict(validation_ds)
true_values = np.array([y for x, y in validation_ds.unbatch()])

r_squared = r2_score(true_values, predictions)
print("R-squared:", r_squared)


Use /var/folders/by/hxzzs0n96hv_9jkkt0cyhdq80000gn/T/tmp1gdodrrg as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.351072. Found 1168 examples.
Training model...
Model trained in 0:00:00.511504
Compiling model...


[INFO 24-03-15 16:21:45.1339 CET kernel.cc:1233] Loading model from path /var/folders/by/hxzzs0n96hv_9jkkt0cyhdq80000gn/T/tmp1gdodrrg/model/ with prefix a6759dceee1642da
[INFO 24-03-15 16:21:45.2557 CET decision_forest.cc:660] Model loaded with 300 root(s), 111010 node(s), and 76 input feature(s).
[INFO 24-03-15 16:21:45.2557 CET abstract_model.cc:1344] Engine "RandomForestOptPred" built
[INFO 24-03-15 16:21:45.2557 CET kernel.cc:1061] Use fast generic engine


Model compiled.
1/1 [==============================] - 0s 84ms/step
R-squared: 0.8555146612256113


In [25]:
test = pd.read_csv("../data/test.csv")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test, task=tfdf.keras.Task.REGRESSION)
test_predictions = model.predict(test_ds)

predictions_df = pd.DataFrame(test_predictions, columns=["SalePrice"])

predictions_df.insert(0, 'Id', test['Id'])

predictions_df.to_csv('../tfdf_predictions.csv', index=False)

2/2 [==============================] - 0s 12ms/step
